In [1]:
import pandas as pd
import numpy as np

# 让输出更友好一点
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_colwidth", 200)

# 1. 读取 Excel
file_path = "step3_private_undergrad_metrics_2025.xlsx"  # 如路径不同，请改成你的实际路径
df = pd.read_excel(file_path)

# 2. 基本信息
print("数据维度：", df.shape)   # (行数, 列数)

print("\n前 5 行预览：")
display(df.head())

print("\n字段信息（dtype）：")
df.info()


数据维度： (417, 147)

前 5 行预览：


,school_name,year,school_code,governing_body,location,level,note,official_site,info_url_1,info_url_2,info_url_3,disclosure_url_1,disclosure_url_2,employment_url_1,employment_url_2,employment_url_3,intl_coop_url_1,intl_coop_url_2,intl_coop_url_3,studyabroad_url_1,studyabroad_url_2,studyabroad_url_3,overseas_practice_url_1,overseas_practice_url_2,asean_url_1,asean_url_2,major_program_url_1,major_program_url_2,school_full_name,school_short_name,english_name,school_type,school_level,school_category,supervising_dept,province,city,founded_year,campus_count,students_total,undergrads_total,junior_students_total,postgrads_total,international_students,annual_new_enrollment,teachers_total,fulltime_teachers,professors_count,associate_professors_count,phd_teachers_count,master_teachers_count,student_teacher_ratio,college_count,department_count,major_count,major_language_related,major_business_related,national_first_class_majors,provincial_first_class_majors,campus_area_mu,campus_area_m2,building_area_m2,library_books,library_ebooks,labs_count,training_bases_count,off_campus_bases_count,intl_partner_universities_count,intl_partner_countries_count,intl_exchange_programs_count,intl_double_degree_programs_count,studyabroad_students_annual,overseas_internship_programs_count,asean_partner_count,asean_programs_count,languages_offered_count,languages_list,employment_rate_2024,employment_rate_2025,further_study_rate_2025,top_employment_regions_2025,top_employment_industries_2025,top_employment_employers_2025,employment_quality_index,profile_page_url,profile_text_snippet,intl_page_url,intl_text_snippet,employment_page_url,employment_text_snippet,asean_page_url,asean_text_snippet,positioning_keywords,intl_keywords,employment_keywords,asean_keywords,metrics_status,intl_status,employment_status,asean_status,last_crawled_at,last_intl_crawled_at,last_employment_crawled_at,last_asean_crawled_at,manual_notes,founded_year_evidence,founded_year_confidence,students_total_evidence,students_total_confidence,junior_students_total_evidence,junior_students_total_confidence,international_students_evidence,international_students_confidence,teachers_total_evidence,teachers_total_confidence,fulltime_teachers_evidence,fulltime_teachers_confidence,phd_teachers_count_evidence,phd_teachers_count_confidence,master_teachers_count_evidence,master_teachers_count_confidence,student_teacher_ratio_evidence,student_teacher_ratio_confidence,major_count_evidence,major_count_confidence,major_language_related_evidence,major_language_related_confidence,major_business_related_evidence,major_business_related_confidence,national_first_class_majors_evidence,national_first_class_majors_confidence,provincial_first_class_majors_evidence,provincial_first_class_majors_confidence,campus_count_evidence,campus_count_confidence,campus_area_mu_evidence,campus_area_mu_confidence,campus_area_m2_evidence,campus_area_m2_confidence,library_books_evidence,library_books_confidence,library_ebooks_evidence,library_ebooks_confidence,off_campus_bases_count_evidence,off_campus_bases_count_confidence,studyabroad_students_annual_evidence,studyabroad_students_annual_confidence
0,三亚学院,2025,4146013892,海南省教育厅,三亚市,本科,民办,https://www.sanyau.edu.cn/,https://www.sanyau.edu.cn/xxgk/,https://www.sanyau.edu.cn/539/list.htm,NaN,https://www.sanyau.edu.cn/xxgk/,NaN,https://www.sanyau.edu.cn/,NaN,NaN,https://www.sanyau.edu.cn/526/list.htm,http://id.sanyau.edu.cn:9092/rsccms/sites/sywsc/wscindex.jsp,NaN,https://www.sanyau.edu.cn/jwglxt/list.htm,NaN,NaN,NaN,NaN,https://www.sanyau.edu.cn/2025/1127/c2605a407654/page.htm,https://www.sanyau.edu.cn/2024/1129/c572a385917/page.htm,http://wy.sanyau.edu.cn/,https://www.sanyau.edu.cn/596/list.htm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.0,NaN,18000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,3.0,NaN,NaN,NaN,NaN,3000.0,2000000.0,NaN,2750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ok,NaN


字段信息（dtype）：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417 entries, 0 to 416
Columns: 147 entries, school_name to studyabroad_students_annual_confidence
dtypes: float64(72), int64(2), object(73)
memory usage: 479.0+ KB


In [2]:
# 计算每一列的缺失比例
missing_ratio = df.isna().mean().sort_values(ascending=False)

print("缺失率最高的前 30 个字段：")
display(missing_ratio.head(30))


缺失率最高的前 30 个字段：


intl_exchange_programs_count       1.0
intl_partner_countries_count       1.0
intl_partner_universities_count    1.0
top_employment_employers_2025      1.0
top_employment_regions_2025        1.0
postgrads_total                    1.0
city                               1.0
professors_count                   1.0
associate_professors_count         1.0
annual_new_enrollment              1.0
undergrads_total                   1.0
province                           1.0
school_full_name                   1.0
english_name                       1.0
school_short_name                  1.0
school_category                    1.0
school_level                       1.0
supervising_dept                   1.0
school_type                        1.0
asean_status                       1.0
employment_status                  1.0
intl_status                        1.0
asean_keywords                     1.0
employment_keywords                1.0
intl_keywords                      1.0
positioning_keywords     

In [3]:
# 自动识别字段类型

numeric_like_cols = []
text_evidence_cols = []
status_cols = []

for col in df.columns:
    c = col.lower()
    
    # 1. 识别证据字段
    if c.endswith("_evidence"):
        text_evidence_cols.append(col)
        continue

    # 2. 识别状态/置信度字段
    if c.endswith("_status") or c.endswith("_confidence"):
        status_cols.append(col)
        continue

    # 3. 数值字段（以 _value 结尾 or 本身看起来是数量）
    if c.endswith("_value") or any(keyword in c for keyword in ["count", "total", "rate", "number"]):
        numeric_like_cols.append(col)

# 输出结果
print("识别到的疑似数值字段（numeric_like_cols）：\n", numeric_like_cols[:30], "... 共", len(numeric_like_cols), "个\n")
print("识别到的 evidence 文本字段（text_evidence_cols）：\n", text_evidence_cols[:10], "... 共", len(text_evidence_cols), "个\n")
print("识别到的状态字段（status_cols）：\n", status_cols[:10], "... 共", len(status_cols), "个\n")


识别到的疑似数值字段（numeric_like_cols）：
 ['campus_count', 'students_total', 'undergrads_total', 'junior_students_total', 'postgrads_total', 'teachers_total', 'professors_count', 'associate_professors_count', 'phd_teachers_count', 'master_teachers_count', 'college_count', 'department_count', 'major_count', 'labs_count', 'training_bases_count', 'off_campus_bases_count', 'intl_partner_universities_count', 'intl_partner_countries_count', 'intl_exchange_programs_count', 'intl_double_degree_programs_count', 'overseas_internship_programs_count', 'asean_partner_count', 'asean_programs_count', 'languages_offered_count', 'employment_rate_2024', 'employment_rate_2025', 'further_study_rate_2025'] ... 共 27 个

识别到的 evidence 文本字段（text_evidence_cols）：
 ['founded_year_evidence', 'students_total_evidence', 'junior_students_total_evidence', 'international_students_evidence', 'teachers_total_evidence', 'fulltime_teachers_evidence', 'phd_teachers_count_evidence', 'master_teachers_count_evidence', 'student_teacher_r

In [4]:
df[numeric_like_cols].dtypes

campus_count                          float64
students_total                        float64
undergrads_total                      float64
junior_students_total                 float64
postgrads_total                       float64
teachers_total                        float64
professors_count                      float64
associate_professors_count            float64
phd_teachers_count                    float64
master_teachers_count                 float64
college_count                         float64
department_count                      float64
major_count                           float64
labs_count                            float64
training_bases_count                  float64
off_campus_bases_count                float64
intl_partner_universities_count       float64
intl_partner_countries_count          float64
intl_exchange_programs_count          float64
intl_double_degree_programs_count     float64
overseas_internship_programs_count    float64
asean_partner_count               

In [5]:
# 查看 evidence 字段的前若干条文本，确认格式
for col in text_evidence_cols:
    print("\n-----", col, "-----")
    print(df[col].dropna().head(3).tolist())



----- founded_year_evidence -----
['三亚学院始建于2005年', '上海中侨职业技术大学（简称“中侨大学”）创建于1993年', '始建于2001年4月']

----- students_total_evidence -----
['学校累计为社会输送近九万名毕业生', '目前，学校拥有全日制在校生18000余人', '全日制本科在校生超万人']

----- junior_students_total_evidence -----
['其中职业本科学生7329人', '开设41个专科专业', '专科1282人']

----- international_students_evidence -----
['目前在校全日制留学生近40人', '累计培养了来自四大洲14个国家的50余名留学生。', '目前，学院已有400余人赴英、法、美、日、韩、芬、德、西班牙等国家进行学习交流']

----- teachers_total_evidence -----
['师资队伍力量雄厚，汇聚了享受国务院特殊津贴专家、全国教学名师等高水平教师', '学校现有专任教师近600人', '教职员工数（人）1552']

----- fulltime_teachers_evidence -----
['其中专任教师1231人', '专任教师626人', '专任教师764人']

----- phd_teachers_count_evidence -----
['230余人拥有或者正在攻读国内外博士学位', '博士学位191人', '具有博士研究生学位教师42人']

----- master_teachers_count_evidence -----
['硕士学位863人', '具有硕士及以上学位教师330人', '硕士研究生及以上学历人员431人']

----- student_teacher_ratio_evidence -----
['在校生11246人，教师近600人，生师比约为19:1', '生师比（24182/1552）约为15.6:1', '生师比接近为19.54:1']

----- major_count_evidence -----
['开设73个本科专业', '设有10个二级学院，24个本科专业（方向）', '现开设英语和国

In [6]:
import re
import numpy as np
from cn2an import cn2an  # 中文数字转阿拉伯数字（若没有 pip install cn2an）

def extract_num_from_text(text, prefer_large=True):
    """
    从 evidence 文本中提取“数量方向”的数字。
    支持：中文数字、万、余、近、多、范围、比例、生师比、亩→㎡ 等。
    prefer_large=True 表示优先选择较大的数字（如多数字时：国家数 vs 人数）
    """
    if text is None or (isinstance(text, float) and np.isnan(text)):
        return np.nan

    t = str(text)

    # 特殊：生师比 19:1
    ratio_match = re.search(r"(\d+(\.\d+)?):(\d+(\.\d+)?)", t)
    if ratio_match:
        try:
            a = float(ratio_match.group(1))
            b = float(ratio_match.group(3))
            return a / b
        except:
            pass

    # 中文数字 → 阿拉伯数字
    try:
        t = re.sub(r"([零一二三四五六七八九十百千万亿两]+)", 
                   lambda m: str(cn2an(m.group(1), "smart")), t)
    except:
        pass

    # 范围文本（取平均）
    range_match = re.search(r"(\d+\.?\d*)\s*[-~–—]\s*(\d+\.?\d*)", t)
    if range_match:
        a = float(range_match.group(1))
        b = float(range_match.group(2))
        return (a + b) / 2

    # 提取所有数字
    nums = re.findall(r"\d+\.?\d*", t)
    if not nums:
        return np.nan

    nums = [float(n) for n in nums]

    # 单位：万（包括“万余人”“万名”）
    if "万" in t:
        nums = [n * 10000 for n in nums]

    # 单位：亩 → 平方米
    if "亩" in t:
        nums = [n * 666.6667 for n in nums]

    # 处理模糊词：近、余、多、左右
    if any(x in t for x in ["近", "余", "多", "左右"]):
        # 取第一个数字
        return nums[-1] if prefer_large else nums[0]

    # 多数字场景
    if len(nums) > 1:
        # 处理：来自20个国家的50名留学生 → 应取较大数字
        if prefer_large:
            return max(nums)
        return nums[0]  # 对于某些字段可能取第一个数字

    return nums[0]


In [7]:
# 为所有 evidence 字段提取数字，生成 *_extracted 字段
for col in text_evidence_cols:
    new_col = col.replace("_evidence", "_extracted")
    df[new_col] = df[col].apply(lambda x: extract_num_from_text(x))

print("已完成数值提取，示例：")
df[[text_evidence_cols[0], text_evidence_cols[0].replace("_evidence", "_extracted")]].head()


已完成数值提取，示例：


,founded_year_evidence,founded_year_extracted
0,三亚学院始建于2005年,2005.0
1,上海中侨职业技术大学（简称“中侨大学”）创建于1993年,1993.0
2,始建于2001年4月,2001.0
3,2004年批准设立的上海首批全日制本科独立学院,2004.0
4,上海师范大学天华学院（Shanghai Normal University TIANHUA College）于2005年4月经国家教育部批准建校,2005.0


In [8]:
df[['students_total_evidence', 'students_total_extracted']].head(5)
df[['teachers_total_evidence', 'teachers_total_extracted']].head(5)
df[['international_students_evidence', 'international_students_extracted']].head(5)


,international_students_evidence,international_students_extracted
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [9]:
# 看一下几个关键字段的缺失情况（原始 vs extracted）
check_cols = [
    ("students_total", "students_total_extracted"),
    ("teachers_total", "teachers_total_extracted"),
    ("international_students", "international_students_extracted"),
    ("major_count", "major_count_extracted"),
]

for v, e in check_cols:
    print(f"\n=== {v} vs {e} ===")
    print("value 缺失数：", df[v].isna().sum())
    print("extracted 提取数：", df[e].notna().sum())



=== students_total vs students_total_extracted ===
value 缺失数： 128
extracted 提取数： 281

=== teachers_total vs teachers_total_extracted ===
value 缺失数： 196
extracted 提取数： 201

=== international_students vs international_students_extracted ===
value 缺失数： 388
extracted 提取数： 28

=== major_count vs major_count_extracted ===
value 缺失数： 101
extracted 提取数： 309


In [10]:
# Step 4-D：安全回填逻辑（不会覆盖原始值，不会乱填）

def safe_fill(value_col, extracted_col):
    """
    如果 value 缺失且 extracted 有值 → 回填
    否则保持原状
    """
    return df[value_col].combine_first(df[extracted_col])


pairs = [
    ("students_total", "students_total_extracted"),
    ("teachers_total", "teachers_total_extracted"),
    ("international_students", "international_students_extracted"),
    ("major_count", "major_count_extracted"),
]

for v, e in pairs:
    df[v + "_filled"] = safe_fill(v, e)


# 查看效果
df[[ "students_total", "students_total_extracted", "students_total_filled"]].head()


,students_total,students_total_extracted,students_total_filled
0,18000.0,90000.0,18000.0
1,18000.0,18000.0,18000.0
2,NaN,NaN,NaN
3,10000.0,NaN,10000.0
4,11246.0,11246.0,11246.0


In [11]:
# Step 4-E：可信度校验函数
def validate_extracted_students(row):
    value = row["students_total"]
    extracted = row["students_total_extracted"]
    text = str(row["students_total_evidence"])

    # 1. 原始值存在 → extracted 仅用于校验
    if pd.notna(value):
        # extracted 明显不合理
        if pd.notna(extracted):
            if extracted > value * 2:
                return "extracted_too_large"
            if extracted > 120000:  # 民办本科不可能到12万
                return "extracted_impossible"
            # 关键词排除“累计、毕业生”
            if any(k in text for k in ["累计", "毕业生"]):
                return "extracted_misleading"
        return "ok"

    # 2. value 缺失 → extracted 可能是唯一来源
    if pd.isna(value):
        if pd.isna(extracted):
            return "missing_both"  # 完全缺失
        
        # 检查 extracted 本身的合理性
        if extracted > 120000:
            return "extracted_impossible"
        if any(k in text for k in ["累计", "毕业生"]):
            return "extracted_misleading"

        # 合理 extracted
        return "use_extracted"

    return "unknown"

df["students_total_validation"] = df.apply(validate_extracted_students, axis=1)

df[["students_total", "students_total_extracted", "students_total_validation"]].head()


,students_total,students_total_extracted,students_total_validation
0,18000.0,90000.0,extracted_too_large
1,18000.0,18000.0,ok
2,NaN,NaN,missing_both
3,10000.0,NaN,ok
4,11246.0,11246.0,ok


In [12]:
# Step 4-F：生成最终可信字段 students_total_final

def choose_final_students(row):
    value = row["students_total"]
    extracted = row["students_total_extracted"]
    status = row["students_total_validation"]

    # 情况 1：value 本身有数字 → 最高可信
    if pd.notna(value):
        return value

    # 情况 2：value 缺失、extracted 可用
    if status == "use_extracted":
        return extracted

    # 情况 3：不可信 extracted 或都缺失
    return np.nan

df["students_total_final"] = df.apply(choose_final_students, axis=1)

df[["students_total", "students_total_extracted", "students_total_validation", "students_total_final"]].head(10)


,students_total,students_total_extracted,students_total_validation,students_total_final
0,18000.0,90000.0,extracted_too_large,18000.0
1,18000.0,18000.0,ok,18000.0
2,NaN,NaN,missing_both,NaN
3,10000.0,NaN,ok,10000.0
4,11246.0,11246.0,ok,11246.0
5,24182.0,24182.0,ok,24182.0
6,NaN,NaN,missing_both,NaN
7,16000.0,16000.0,ok,16000.0
8,18000.0,2024.0,ok,18000.0
9,NaN,NaN,missing_both,NaN


In [13]:
for v, e in [
    ("teachers_total", "teachers_total_extracted"),
    ("major_count", "major_count_extracted"),
]:
    print(f"\n=== {v} vs {e} ===")
    print("value 缺失数：", df[v].isna().sum())
    print("extracted 提取数：", df[e].notna().sum())



=== teachers_total vs teachers_total_extracted ===
value 缺失数： 196
extracted 提取数： 201

=== major_count vs major_count_extracted ===
value 缺失数： 101
extracted 提取数： 309


In [14]:
def build_final_numeric(value_col, extracted_col, max_reasonable=None):
    """
    通用规则：
    1）优先使用 value_col
    2）value 缺失时，用 extracted_col 补
    3）如果设置了 max_reasonable，上限之外的值视为不可信，变成 NaN
    """
    final_col = value_col + "_final"

    def choose(row):
        v = row[value_col]
        x = row[extracted_col]

        # 情况 1：value 已有
        if pd.notna(v):
            # 上限校验
            if (max_reasonable is not None) and (v > max_reasonable):
                return np.nan
            return v

        # 情况 2：value 缺失，用 extracted
        if pd.notna(x):
            if (max_reasonable is not None) and (x > max_reasonable):
                return np.nan
            return x

        # 情况 3：都没有
        return np.nan

    df[final_col] = df.apply(choose, axis=1)
    return final_col


In [15]:
# 教师总数：一般民办本科不太可能超过 5000 人，超出就当异常
teachers_final_col = build_final_numeric(
    "teachers_total", "teachers_total_extracted", max_reasonable=5000
)

# 专业总数：不太可能>200个，设个较宽松上限
major_final_col = build_final_numeric(
    "major_count", "major_count_extracted", max_reasonable=200
)

# 看前几行效果
df[["teachers_total", "teachers_total_extracted", teachers_final_col]].head(10)
df[["major_count", "major_count_extracted", major_final_col]].head(10)


,major_count,major_count_extracted,major_count_final
0,3.0,73.0,3.0
1,24.0,24.0,24.0
2,2.0,2.0,2.0
3,29.0,29.0,29.0
4,46.0,46.0,46.0
5,0.0,40.0,0.0
6,NaN,NaN,NaN
7,7.0,17.0,7.0
8,NaN,NaN,NaN
9,8.0,48.0,8.0


In [16]:
# Step 6-A：师生比一致性检查

def check_ratio(row):
    students = row["students_total_final"]
    teachers = row["teachers_total_final"]
    extracted_ratio = row.get("student_teacher_ratio_extracted", np.nan)

    # 缺失情况
    if pd.isna(students) or pd.isna(teachers):
        return "missing"

    # 计算估算师生比
    ratio_calc = students / teachers

    # 若 extracted_ratio 存在，检查一致性
    if pd.notna(extracted_ratio):
        # |ratio_calc - extracted_ratio| / extracted_ratio < 0.25
        if abs(ratio_calc - extracted_ratio) / extracted_ratio < 0.25:
            return "consistent"

    # 检查是否在合理区间
    if ratio_calc < 5:
        return "too_low"
    if ratio_calc > 50:
        return "too_high"

    return "ok"

df["teacher_student_ratio_check"] = df.apply(check_ratio, axis=1)

df[["students_total_final", "teachers_total_final", "student_teacher_ratio_extracted", "teacher_student_ratio_check"]].head(10)


,students_total_final,teachers_total_final,student_teacher_ratio_extracted,teacher_student_ratio_check
0,18000.0,NaN,NaN,missing
1,18000.0,NaN,NaN,missing
2,NaN,NaN,NaN,missing
3,10000.0,NaN,NaN,missing
4,11246.0,600.0,19.00,consistent
5,24182.0,1231.0,15.60,ok
6,NaN,NaN,NaN,missing
7,16000.0,NaN,19.54,missing
8,18000.0,NaN,NaN,missing
9,NaN,1000.0,NaN,missing


In [17]:
# Step 6-B：本科 + 专科 ≈ 总学生数 一致性检查

def check_undergrad_vs_total(row):
    ug = row["undergrads_total"]
    jr = row["junior_students_total"]
    total = row["students_total_final"]

    # 若任何字段缺失
    if pd.isna(total) or pd.isna(ug) or pd.isna(jr):
        return "missing"

    sum_sub = ug + jr

    # 防止除零
    if total == 0:
        return "invalid_total_zero"

    # 偏差率
    diff_ratio = abs(sum_sub - total) / total

    if diff_ratio <= 0.25:
        return "ok"
    else:
        return "inconsistent"

df["undergrad_vs_total_check"] = df.apply(check_undergrad_vs_total, axis=1)

df[["undergrads_total", "junior_students_total", "students_total_final", 
    "undergrad_vs_total_check"]].head(10)


,undergrads_total,junior_students_total,students_total_final,undergrad_vs_total_check
0,NaN,NaN,18000.0,missing
1,NaN,NaN,18000.0,missing
2,NaN,NaN,NaN,missing
3,NaN,NaN,10000.0,missing
4,NaN,NaN,11246.0,missing
5,NaN,NaN,24182.0,missing
6,NaN,NaN,NaN,missing
7,NaN,NaN,16000.0,missing
8,NaN,NaN,18000.0,missing
9,NaN,NaN,NaN,missing


In [18]:
# 看看有哪些面积相关的 extracted 字段
[c for c in df.columns if "campus_area" in c and "extracted" in c]


['campus_area_mu_extracted', 'campus_area_m2_extracted']

In [19]:
# 优先用 m2_extracted，没有就用 mu_extracted
df["campus_area_m2_final"] = df["campus_area_m2_extracted"].combine_first(
    df["campus_area_mu_extracted"]
)

df[["campus_area_m2_extracted", "campus_area_mu_extracted", "campus_area_m2_final"]].head(10)


,campus_area_m2_extracted,campus_area_mu_extracted,campus_area_m2_final
0,2.000000e+06,2.000000e+06,2.000000e+06
1,NaN,NaN,NaN
2,2.310000e+05,2.310000e+05,2.310000e+05
3,NaN,NaN,NaN
4,3.960000e+05,3.960000e+05,3.960000e+05
5,5.325000e+05,5.333334e+09,5.325000e+05
6,NaN,NaN,NaN
7,1.381907e+04,NaN,1.381907e+04
8,NaN,NaN,NaN
9,8.186667e+05,8.186667e+05,8.186667e+05


In [20]:
# 人均校园面积（㎡/人）
df["area_per_student_m2"] = df["campus_area_m2_final"] / df["students_total_final"]

df[["campus_area_m2_final", "students_total_final", "area_per_student_m2"]].head(10)


,campus_area_m2_final,students_total_final,area_per_student_m2
0,2.000000e+06,18000.0,111.111117
1,NaN,18000.0,NaN
2,2.310000e+05,NaN,NaN
3,NaN,10000.0,NaN
4,3.960000e+05,11246.0,35.212522
5,5.325000e+05,24182.0,22.020511
6,NaN,NaN,NaN
7,1.381907e+04,16000.0,0.863692
8,NaN,18000.0,NaN
9,8.186667e+05,NaN,NaN


In [21]:
def check_area_per_student(row):
    area = row["campus_area_m2_final"]
    students = row["students_total_final"]
    if pd.isna(area) or pd.isna(students) or students == 0:
        return "missing"

    aps = area / students  # area per student

    if aps < 5:
        return "too_crowded"        # 极度拥挤（疑似数据错误）
    elif aps < 15:
        return "crowded"            # 偏拥挤
    elif aps <= 60:
        return "normal"             # 正常
    elif aps <= 200:
        return "spacious"           # 宽裕
    else:
        return "too_large"          # 过大，疑似数据问题

df["campus_area_check"] = df.apply(check_area_per_student, axis=1)

df[["campus_area_m2_final", "students_total_final", "area_per_student_m2", "campus_area_check"]].head(10)


,campus_area_m2_final,students_total_final,area_per_student_m2,campus_area_check
0,2.000000e+06,18000.0,111.111117,spacious
1,NaN,18000.0,NaN,missing
2,2.310000e+05,NaN,NaN,missing
3,NaN,10000.0,NaN,missing
4,3.960000e+05,11246.0,35.212522,normal
5,5.325000e+05,24182.0,22.020511,normal
6,NaN,NaN,NaN,missing
7,1.381907e+04,16000.0,0.863692,too_crowded
8,NaN,18000.0,NaN,missing
9,8.186667e+05,NaN,NaN,missing


In [22]:
# Step 4 Output：导出清洗后的数据

clean_cols = [
    "school_name",

    # 最终核心指标
    "students_total_final",
    "teachers_total_final",
    "major_count_final",
    "campus_area_m2_final",

    # 重要辅助字段（可选，用于质量检查）
    "area_per_student_m2",
    "teacher_student_ratio_check",
    "undergrad_vs_total_check",
    "campus_area_check",

    # 原始值（回溯用）
    "students_total",
    "students_total_extracted",
    "teachers_total",
    "teachers_total_extracted",
    "major_count",
    "major_count_extracted",
    "campus_area_m2_extracted",
    "campus_area_mu_extracted"
]

df_cleaned = df[clean_cols]

df_cleaned.to_excel("step4_cleaned_metrics_2025.xlsx", index=False)

df_cleaned.head()


,school_name,students_total_final,teachers_total_final,major_count_final,campus_area_m2_final,area_per_student_m2,teacher_student_ratio_check,undergrad_vs_total_check,campus_area_check,students_total,students_total_extracted,teachers_total,teachers_total_extracted,major_count,major_count_extracted,campus_area_m2_extracted,campus_area_mu_extracted
0,三亚学院,18000.0,NaN,3.0,2.000000e+06,111.111117,missing,missing,spacious,18000.0,90000.0,NaN,NaN,3.0,73.0,2.000000e+06,2.000000e+06
1,上海中侨职业技术大学,18000.0,NaN,24.0,NaN,NaN,missing,missing,missing,18000.0,18000.0,NaN,NaN,24.0,24.0,NaN,NaN
2,上海兴伟学院,NaN,NaN,2.0,2.310000e+05,NaN,missing,missing,missing,NaN,NaN,NaN,NaN,2.0,2.0,2.310000e+05,2.310000e+05
3,上海外国语大学贤达经济人文学院,10000.0,NaN,29.0,NaN,NaN,missing,missing,missing,10000.0,NaN,NaN,NaN,29.0,29.0,NaN,NaN
4,上海师范大学天华学院,11246.0,600.0,46.0,3.960000e+05,35.212522,consistent,missing,normal,11246.0,11246.0,600.0,600.0,46.0,46.0,3.960000e+05,3.960000e+05


In [25]:
import pandas as pd

# 载入两个文件
df_step3 = pd.read_excel("E:/gwydata/pythonProject/step3_private_undergrad_metrics_2025.xlsx")
df_clean = pd.read_excel("E:/gwydata/pythonProject/step4_cleaned_metrics_2025.xlsx")

# 检查关键主键（必须一致）
print(df_step3.shape, df_clean.shape)
print(df_step3["school_name"].nunique(), df_clean["school_name"].nunique())

# 以 school_name 为主键合并
df_merged = df_step3.merge(df_clean, on="school_name", how="left", suffixes=("", "_cleaned"))

# 保存最终整合文件
output_path = "E:/gwydata/pythonProject/step4_merged_full_metrics_2025.xlsx"
df_merged.to_excel(output_path, index=False)

output_path


(417, 147) (417, 17)
417 417


'E:/gwydata/pythonProject/step4_merged_full_metrics_2025.xlsx'